# 读入库

In [1]:
import jiagu
import string
import pandas as pd
import torch
from PIL import Image
import numpy as  np
import cn_clip.clip as clip
from cn_clip.clip import load_from_name, available_models
import itertools
import jiagu
import string
import re
import os
import fitz # PyMuPDF
import pdb
import ast
import math

from tqdm import tqdm

print("Available models:", available_models())
# Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']

/Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Available models: ['ViT-B-16', 'ViT-L-14', 'ViT-L-14-336', 'ViT-H-14', 'RN50']


# 读入文本和图片信息的DataFrame

In [26]:
# 读入文本DataFrame
df_text = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/01 text/text_df_3.xlsx")
# df_text = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/03_TextImgMatch/partly/文本提取_text_df_2.xlsx")

# 读入图片DataFrame
df_img = pd.read_excel("/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/03 PyMu_img01_coordinate/cord_final2.xlsx")

# 函数定义
## 去除字段中的标点和空格

In [4]:
def remove_punctuation_and_spaces(text):
    # 去除标点符号
    text = str(text)
    text = text.translate(str.maketrans("", "", string.punctuation))

    # 去除空格
    text = re.sub(r"\s+", "", text)

    return text

## 从文本段中提取关键词

In [5]:
# fin = open('GS.txt', mode='r',encoding='UTF-8')
# text = fin.read()
# fin.close()

def labelization(text, key_num = 3):
#     summarize = jiagu.summarize(text, 1) # 摘要
#     print("Sum:", summarize)
#     keywords = jiagu.keywords(text, 15) # 关键词
#     print("\nKeyWords:", keywords)

    cleaned_sentence = remove_punctuation_and_spaces(text)
#     print("\nCleaned Sentence:", cleaned_sentence)
    # summarize1 = jiagu.summarize(cleaned_sentence, 1) # 摘要
    # print(summarize1)
    keywords1 = jiagu.keywords(cleaned_sentence, key_num) # 关键词
#     print("KeyWords:", keywords1,"\n\n")
    
    return keywords1
# fin = open('input.txt', 'r')
# text = fin.read()
# fin.close()
#
# summarize = jiagu.summarize(text, 3) # 摘要
# print(summarize)

## 提取当页全部关键词

In [6]:
def extract_keywords(PDF_name, page):
    keywords_list = []
    keyword = []
#     page = page + 1
    
#     unique_pages = df_text[df_text["PDF_name"] == PDF_name]['page'].unique()
#     for page in unique_pages:
#         # 指定要获取的行
    page_rows = df_text[(df_text["PDF_name"] == PDF_name) & (df_text['page'] == page)]


    for idx, row in page_rows.iterrows():
        # 提取关键词
        if not pd.isna(row['content']):
            keywords = labelization(row['content'], len(str(row['content'])) // 20 + 2)
#         print(keywords)
        # 将关键词保存至特定行的'keyword'列
        df_text['keyword'] = df_text['keyword'].astype('object')
#         df_text.loc[idx, "keyword"] = keywords
        df_text.at[idx, 'keyword'] = keywords

        # 加入当前页的总'keyword_list'
        keywords_list.append(keywords)
        keyword = list(itertools.chain.from_iterable(keywords_list))
    
    return keyword

## 获取特定图片在当页标签中的概率

In [7]:
# img_path = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf_page_10_img_1.png"

# keywords = [keyword for keyword in keywords_list if pattern.match(keyword)]

def jiagu_prob(img_path, keywords):
    # 预处理，读入模型
    device='cpu'
    model, preprocess = load_from_name("RN50", device=device, download_root='./')
    model.eval()

    # 转移到mps上进行gpu加速
    device = "mps" if torch.backends.mps.is_available() == True else 'cpu'
    model = model.to(device)

    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    label = clip.tokenize(keywords).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(label)
        # 对特征进行归一化，用于下游任务
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)

        logits_per_image, logits_per_text = model.get_similarity(image, label)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    return probs, img_path

# print("\nLabel probs:", probs)  # [[1.268734e-03 5.436878e-02 6.795761e-04 9.436829e-01]]
# labels = keywords
# max_prob_index = np.argmax(probs)
# max_prob_label = labels[max_prob_index]
# print("\nMax probability label:", max_prob_label)

## 图片匹配文字

In [8]:
def text_match_img(labels, probs, img_path, PDF_name, page, df_text = df_text, df_img = df_img):
    """
    将每个图片最配对文本保留在df_img中
    将最配对文本的可信度（CRI）保留
    """
    # 创建一个标签到概率的映射字典
    prob_dict = dict(zip(labels, probs[0]))
#     print("\nProb_Dict:\n", prob_dict, "\n\n\n")
    filtered_df = df_text[(df_text["PDF_name"] == PDF_name) & (df_text['page'] == page)]
    if not filtered_df.empty:
        filtered_df = filtered_df[pd.notnull(filtered_df['keyword'])]

        # 创建一个新列来存储每个标签组的概率总和
        filtered_df['prob_sum'] = filtered_df['keyword'].apply(
            lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) / 
                             (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
        )

        
        # 使用 apply() 方法遍历列
        filtered_df['prob_sum'] = filtered_df['prob_sum'].apply(lambda x: 0.0 if not isinstance(x, float) else x)
#         print("\nFiltered_df:\n",filtered_df, "\n")
        
        if not filtered_df['prob_sum'].empty:
            max_prob_content = filtered_df.loc[filtered_df['prob_sum'].idxmax(), 'content']
            max_prob = filtered_df['prob_sum'].max()
            cri = CRI(filtered_df)
        else:
        # 如果 prob_sum 列为空
            max_prob_content = None  # 或任何合适的默认值
            max_prob = None
            cri = None

        # 获取图片名称并存储至df_img
        img_name = os.path.basename(img_path)
        df_img.loc[df_img["file_name"] == img_name, "match_text"] = max_prob_content
        df_img.loc[df_img["file_name"] == img_name, "match_text_prob"] = max_prob
        df_img.loc[df_img["file_name"] == img_name, "match_text_CRI"] = cri
        
#         print(max_prob_content, max_prob, cri)
        
        return max_prob_content

In [ ]:
# beta
def text_match_img(labels, probs, img_path, PDF_name, page, df_text = df_text, df_img = df_img):
    """
    将每个图片最配对文本保留在df_img中
    将最配对文本的可信度（CRI）保留
    """
    # 创建一个标签到概率的映射字典
    prob_dict = dict(zip(labels, probs[0]))
#     print("\nProb_Dict:\n", prob_dict, "\n\n\n")

    # 找到物理最近的图文
    img_name = os.path.basename(img_path)
    img_cord = df_img[df_img['file_name'] == img_name]['centre_coordinate']
    [img_cord] = img_cord.values
    img_cord = ast.literal_eval(img_cord)
    
    df_min_dis = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page)]
    
    if not df_min_dis.empty:
        df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)
        
        return df_min_dis.sort_values(by='dis_to_img_current', ascending = True)[0:1]['keyword'].apply(
               lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) /
                         (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
            )
    else:
        print("{0:=^40}".format('Anomaly Founded'))
        print("PDF:{} has no text content on page{}".format(PDF_name, page))
        return -1
    
    

    # 配对图片与文字
    filtered_df = df_text[(df_text["PDF_name"] == PDF_name) & (df_text['page'] == page)]
    if not filtered_df.empty:
        filtered_df = filtered_df[pd.notnull(filtered_df['keyword'])]

        # 创建一个新列来存储每个标签组的概率总和
        filtered_df['prob_sum'] = filtered_df['keyword'].apply(
            lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) / 
                             (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
        )

        
        # 使用 apply() 方法遍历列
        filtered_df['prob_sum'] = filtered_df['prob_sum'].apply(lambda x: 0.0 if not isinstance(x, float) else x)
#         print("\nFiltered_df:\n",filtered_df, "\n")
        
        if not filtered_df['prob_sum'].empty:
            max_prob_content = filtered_df.loc[filtered_df['prob_sum'].idxmax(), 'content']
            max_prob = filtered_df['prob_sum'].max()
            cri = CRI(filtered_df)
        else:
        # 如果 prob_sum 列为空
            max_prob_content = None  # 或任何合适的默认值
            max_prob = None
            cri = None

        # 存储至df_img
        df_img.loc[df_img["file_name"] == img_name, "match_text"] = max_prob_content
        df_img.loc[df_img["file_name"] == img_name, "match_text_prob"] = max_prob
        df_img.loc[df_img["file_name"] == img_name, "match_text_CRI"] = cri
        
        return max_prob_content

## 计算标签组概率的可靠性

In [9]:
def CRI(filtered_df):
    """
    综合可靠性指数(Composite Reliability Index, CRI)
        计算: 最大概率差异度(Maximum Probability Spread, MPS) / 标准差(Standard Deviation, SD)
    
    最大概率差异度(Maximum Probability Spread, MPS): 
        计算: 最大概率值与其他概率值的差值，然后取平均值。
    """
    # 计算MPS和SD
    max_probability = filtered_df['prob_sum'].max()  # 最大概率值
    average_probability = filtered_df['prob_sum'].mean()  # 所有概率的平均值
    probabilities_except_max = filtered_df[filtered_df['prob_sum'] != max_probability]['prob_sum']  # 除了最大值的所有概率值

    # 计算MPS
    mps = max_probability - probabilities_except_max.mean() if not probabilities_except_max.empty else 0

    # 计算标准差
    sd = filtered_df['prob_sum'].std()  # 所有概率值的标准差

    # 计算CRI
    cri = mps / sd if sd > 0 else 0  # 如果标准差为0，则CRI没有意义
    
    return cri

## 一次获取PDF中每一页的全部标签

In [ ]:
# def label_in_PDF(PDF_path):
#     pdf_file = fitz.open(PDF_path)
#     pdf_name = os.path.basename(PDF_path)

#     key = []

#     # 遍历PDF的每一页
#     for page in tqdm(range(len(pdf_file))):
#         keywords = extract_keywords(pdf_name, page)
#     #     print("Page:{}\n{}\n\n".format(page,keywords))
#         key.append({'file_name': pdf_name,
#                     'page': page + 1,
#                     'keywords': keywords})

#     # 创建临时df存放每一页的关键词组 
#     df_key = pd.DataFrame(key)
    
#     return df_key

In [10]:
# beta
def label_in_PDF(PDF_path):
    pdf_file = fitz.open(PDF_path)
    pdf_name = os.path.basename(PDF_path)

    key = []

    # 遍历PDF的每一页
    for page in tqdm(range(len(pdf_file))):
        keywords = extract_keywords(pdf_name, page+1)
    #     print("Page:{}\n{}\n\n".format(page,keywords))
        key.append({'file_name': pdf_name,
                    'page': page + 1,
                    'keywords': keywords})

    # 创建临时df存放每一页的关键词组 
    df_key = pd.DataFrame(key)
    
    return df_key

## 计算距离

In [11]:
def distance(img_cord, text_cord):
    """
    Takes in both 'img_cord' and 'text_cord' in tuple of (x, y).
    Return the distance between two cord.
    """
    img_x, img_y = img_cord
    text_x, text_y = text_cord
    
    return math.sqrt((text_x - img_x)**2 + (text_y - img_y)**2)

In [12]:
def get_bundle_path(directory):
    files = [f for f in os.listdir(directory) if f != '.DS_Store']
    return [os.path.join(directory, file) for file in files if os.path.isfile(os.path.join(directory, file))]

# 计算最近的图文间的语义匹配度

In [13]:
def wording_similarity(labels, probs, img_path, PDF_name, page, df_text = df_text, df_img = df_img):
    # 创建一个标签到概率的映射字典
    prob_dict = dict(zip(labels, probs[0]))
    
    # 获得图片坐标
    img_name = os.path.basename(img_path)
    img_cord = df_img[df_img['file_name'] == img_name]['centre_coordinate']
    [img_cord] = img_cord.values
    img_cord = ast.literal_eval(img_cord)
        
    # 获取文本段坐标
    df_min_dis = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page)]
#     df_min_dis = df_min_dis(pd.notnull(df_min_dis['keyword']))
    if not df_min_dis.empty:
        df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)
        
        # 计算标签组概率
        return df_min_dis.sort_values(by='dis_to_img_current', ascending = True)[0:1]['keyword'].apply(
               lambda keywords: sum(prob_dict.get(k, 0) for k in keywords if pd.notnull(k)) /
                         (len([k for k in keywords if pd.notnull(k)]) if keywords else 1)
            )
    else:
        print("{0:=^40}".format('Anomaly Founded'))
        print("PDF:{} has no text content on page{}".format(PDF_name, page))
        return -1

In [19]:
img_path = '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf_page_10_img_1.png'
PDF_path = '/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf'
df_key = label_in_PDF(PDF_path)
[keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == 10)]['keywords'])
[keywords_t1] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == 8)]['keywords'])

probs, img_path = jiagu_prob(img_path, keywords)

text_match_img(keywords, probs, img_path, os.path.basename(PDF_path), page=10)
ws = wording_similarity(keywords, probs, img_path, os.path.basename(PDF_path), page=10)

print(ws, type(ws))

100%|███████████████████████████████████████████| 50/50 [00:03<00:00, 13.68it/s]


Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
2190    0.071095
Name: keyword, dtype: float64 <class 'pandas.core.series.Series'>


/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_5698/2785949079.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)


# 运行

In [ ]:
# 指定要处理的PDF路径
# PDF_path = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022/00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf"

In [14]:
# 指定要处理的PDF文件夹
# folder_dir = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/00_DataBase/00_SustainabilityReport_PDF/2022"
folder_dir = "/Users/improvise/Downloads/test"

# 指定要处理的图片文件夹
img_folder = "/Users/improvise/Desktop/保研/实证论文/ESG/Playground/01_Extraction/Test01/02 PyMu_img01/"

In [27]:
PDF_path_list = get_bundle_path(folder_dir)

for PDF_path in PDF_path_list:
    df_key = label_in_PDF(PDF_path)
    for row in df_img.loc[df_img['PDF_name']==os.path.basename(PDF_path)].itertuples():
        img_name = row.file_name
        page = row.page
        p_index = row.p_index
        img_cord = row.centre_coordinate
#         print('\n\n', page, p_index, '\n\n')
        
        # 读图
        img_path = os.path.join(img_folder, img_name)
        # 读页面关键词
        [keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == page)]['keywords'])
        [keywords_t1] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == page +1)]['keywords'])
#         print('Keywords:\n', keywords,'\n')
#         print('Keywords_t1:\n', keywords_t1, '\n\n\n')
        
        # 计算概率
        probs, img_path = jiagu_prob(img_path, keywords)

        # 计算语义近似度
        ws = wording_similarity(keywords_t1, probs, img_path, os.path.basename(PDF_path), page=page)
        if isinstance(ws, pd.Series):
            [ws] = list(ws) 
        df_img.loc[df_img["file_name"] == img_name, "word_simi"] = ws
        print("\n\nWording Similarity:\n", ws)
        
        # 匹配图文
        object_text = text_match_img(keywords, probs, img_path, os.path.basename(PDF_path), page)
        if object_text != None:
        # 获取文本坐标
            text_cord_list = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page) & (df_text['content'] == object_text)][['center_x','center_y']].values.tolist()

            if text_cord_list:
                text_cord = text_cord_list[0]  # 选择第一个元素

            # 接下来的计算
            else:
                [text_cord] = text_cord_list

            # 计算距离
            if type(img_cord) == str:
                    img_cord = ast.literal_eval(img_cord)

            dist = distance(img_cord, text_cord)
            df_img.loc[df_img["file_name"] == img_name, "distance"] = dist

100%|███████████████████████████████████████████| 50/50 [00:03<00:00, 13.11it/s]


Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}
============Anomaly Founded=============
PDF:00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf has no text content on page1


Wording Similarity:
 -1
Loading vision model config from /Users/improvise/miniconda/env

/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_5698/2785949079.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)




Wording Similarity:
 0.0
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}


/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_5698/2785949079.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)


...



Wording Similarity:
 0.004439005453605205
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}


/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_5698/2785949079.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)




Wording Similarity:
 0.0
Loading vision model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RN50.json
Loading text model config from /Users/improvise/miniconda/envs/Grace/lib/python3.9/site-packages/Chinese-CLIP-master/cn_clip/clip/model_configs/RBT3-chinese.json
Model info {'embed_dim': 1024, 'image_resolution': 224, 'vision_layers': [3, 4, 6, 3], 'vision_width': 64, 'vision_patch_size': None, 'vocab_size': 21128, 'text_attention_probs_dropout_prob': 0.1, 'text_hidden_act': 'gelu', 'text_hidden_dropout_prob': 0.1, 'text_hidden_size': 768, 'text_initializer_range': 0.02, 'text_intermediate_size': 3072, 'text_max_position_embeddings': 512, 'text_num_attention_heads': 12, 'text_num_hidden_layers': 3, 'text_type_vocab_size': 2}


Wording Similarity:
 4.765717312693596e-05


/var/folders/kd/t2wnd4h911v09vtv543hpkg80000gn/T/ipykernel_5698/2785949079.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_min_dis['dis_to_img_current'] = df_min_dis.apply(lambda row: distance(img_cord, (row['center_x'], row['center_y'])), axis = 1)


ValueError: not enough values to unpack (expected 1, got 0)

In [ ]:
# for row in df_img.loc[df_img['PDF_name']==os.path.basename(PDF_path)].itertuples():
#     img_name = row.file_name
#     page = row.page
#     p_index = row.p_index
#     img_cord = row.centre_coordinate
    
#     # 读图
#     img_path = os.path.join(img_folder, img_name)
#     # 读页面关键词
#     [keywords] = list(df_key.loc[(df_key['file_name'] == os.path.basename(PDF_path)) & (df_key['page'] == page)]['keywords'])
#     # 计算概率
#     probs, img_path = jiagu_prob(img_path, keywords)
    
#     # 匹配图文
#     object_text = text_match_img(keywords, probs, img_path, os.path.basename(PDF_path), page)
#     if object_text != None:
#     # 获取文本坐标
#         text_cord_list = df_text.loc[(df_text['PDF_name'] == os.path.basename(PDF_path)) & (df_text['page'] == page) & (df_text['content'] == object_text)][['center_x','center_y']].values.tolist()

#         if text_cord_list:
#             text_cord = text_cord_list[0]  # 选择第一个元素
            
#         # 接下来的计算
#         else:
#             [text_cord] = text_cord_list
        
#         # 计算距离
#         if type(img_cord) == str:
#                 img_cord = ast.literal_eval(img_cord)

#         dist = distance(img_cord, text_cord)
#         df_img.loc[df_img["file_name"] == img_name, "distance"] = dist

In [ ]:
df_img.to_excel("/Users/improvise/Downloads/distance.xlsx", index=False)

In [28]:
df_img.loc[df_img['PDF_name'] == "00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf"]

,Unnamed: 0,PDF_name,page,p_index,file_name,x0,y0,x1,y1,centre_coordinate,word_simi,distance
184,184,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,1,1,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,-1.145007,0.000000,651.968445,807.873962,"(325.41171884536743, 403.9369812011719)",-1.000000,NaN
185,185,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,3,1,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,50.198135,158.740173,373.527161,772.236206,"(211.8626480102539, 465.4881896972656)",0.000000,226.965982
186,186,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,6,1,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,-1.148245,-1.098877,653.037964,809.055359,"(325.9448593258858, 403.9782409667969)",0.013615,643.749598
187,187,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,7,1,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,75.537048,179.527542,308.736328,329.112610,"(192.13668823242188, 254.32007598876953)",0.000000,174.863431
188,188,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,7,2,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,330.382355,178.470428,565.629578,330.169800,"(448.00596618652344, 254.3201141357422)",0.000919,159.737642
...,...,...,...,...,...,...,...,...,...,...,...,...
247,247,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,33,3,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,1020.472473,521.574280,1247.098022,642.646179,"(1133.7852478027344, 582.1102294921875)",0.000417,275.160372
248,248,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,34,1,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,-1.171264,-1.092468,653.046204,808.902527,"(325.93746972084045, 403.905029296875)",0.004439,231.737172
249,249,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,44,1,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,105.125595,170.073380,212.001129,219.733154,"(158.56336212158203, 194.90326690673828)",0.000000,877.032485
250,250,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pdf,44,2,00941.HK-中国移动-中国移动 2022年度可持续发展报告-2023-03-24.pd...,792.624023,570.650391,913.930725,616.882690,"(853.2773742675781, 593.7665405273438)",0.000048,271.848804
